# Training NLU Model

In [1]:
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer, Metadata, Interpreter

In [2]:
def train_nlu(data,configuration,model_dir):
    training_data = load_data(data)
    trainer = Trainer(config.load(configuration))
    trainer.train(training_data)
    model_directory = trainer.persist(model_dir,fixed_model_name='weathernlu')

In [3]:
train_nlu_model_switch = False

In [4]:
if (train_nlu_model_switch):
    train_nlu('./data/data.json','config_spacy.json','./models/nlu')

In [5]:
def run_nlu(query):
    interpreter = Interpreter.load('./models/nlu/default/weathernlu')
    return interpreter.parse(query)

In [6]:
query = "I am planning a holiday to India. I wonder how is the weather out there"

In [7]:
parsed_query = run_nlu(query)
query_intent = parsed_query['intent']['name']
query_entity = parsed_query['entities'][0]['value']
print('Intent: {} \n Entity: {}'.format(query_intent,query_entity))

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/nlu/default/weathernlu\entity_synonyms.json'
  "".format(entity_synonyms_file))


Intent: inform 
 Entity: india


In [8]:
parsed_query

{'entities': [{'confidence': 0.7114679175622723,
   'end': 32,
   'entity': 'location',
   'extractor': 'ner_crf',
   'start': 27,
   'value': 'india'}],
 'intent': {'confidence': 0.80780585954411999, 'name': 'inform'},
 'intent_ranking': [{'confidence': 0.80780585954411999, 'name': 'inform'},
  {'confidence': 0.12270394872887903, 'name': 'goodbye'},
  {'confidence': 0.069490191727000813, 'name': 'greet'}],
 'text': 'I am planning a holiday to India. I wonder how is the weather out there'}

# Custom Action

In [9]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

from rasa_core.actions.action import Action
from rasa_core.events import SlotSet

from apixu.client import ApixuClient

In [10]:
class ActionWeather(Action):
    def name(self):
        return 'action_weather'
    
    def run(self, dispatcher, tracker, domain): 
        api_key = '5ed103a862bd442182385105181306'
        client = ApixuClient(api_key)
        loc = tracker.get_slot('location')
        current = client.getCurrentWeather(q=loc)
        city = current['location']['name']
        condition = current['current']['condition']['text']
        response = """It is currently {} in {} at the moment.""".format(condition,city)
        dispatcher.utter_message(response)
        return [SlotSet('location',loc)]

# Training Dialog Management System

In [11]:
import logging

from rasa_core.agent import Agent
from rasa_core.policies import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy

In [12]:
train_dialog_management_switch = False

In [13]:
if (train_dialog_management_switch):
    logging.basicConfig(level='INFO')
    training_data_file = './data/stories.md'
    model_path = './models/dialogue'
    agent = Agent('weather_domain.yml', policies=[MemoizationPolicy(), KerasPolicy()])
    agent.train(training_data_file,
                augmentation_factor=50,
                #max_history=2,
                epochs=500,
                batch_size=10,
                validation_split=0.2)
    agent.persist(model_path)

# Build Stories Online

In [14]:
from rasa_core.channels.console import ConsoleInputChannel
from rasa_core.interpreter import RegexInterpreter, RasaNLUInterpreter

In [15]:
def run_weather_online(input_channel, interpreter, domain_file="weather_domain.yml", training_data='data/stories.md'):
    agent = Agent(domain_file,
                  policies=[MemoizationPolicy(),KerasPolicy()],
                  interpreter=interpreter)
    agent.train_online(training_data_file,
                       input_channel=input_channel,
                       max_history=2,
                       batch_size=50,
                       epochs=200,
                       max_training_samples=300)
    return agent

In [16]:
build_stories_online_switch = False

In [17]:
if (build_stories_online_switch):
    logging.basicConfig(level="INFO")
    nlu_interpreter = RasaNLUInterpreter('./models/nlu/default/weathernlu')
    run_weather_online(ConsoleInputChannel(), nlu_interpreter)

# Train Dialog Management System from Online Stories

In [18]:
def train_dialogue(domain_file='weather_domain.yml',model_path='./models/dialogue', training_data_file='./data/stories.md'):
    agent = Agent(domain_file, policies=[MemoizationPolicy(), KerasPolicy()])
    agent.train(training_data_file,
                epochs=300,
                batch_size=50,
                validation_split=0.2)
    agent.persist(model_path)
    return agent

In [19]:
def run_weather_bot(serve_forever=True):
    interpreter = RasaNLUInterpreter('./models/nlu/default/weathernlu')
    agent = Agent.load('./models/dialogue',interpreter=interpreter)
    if(serve_forever):
        agent.handle_channel(ConsoleInputChannel())
    return agent

In [20]:
train_dialog_management_online_stories = False

In [21]:
run_weather_bot_switch = True

In [ ]:
if (train_dialog_management_online_stories):
    train_dialogue()

In [ ]:
if (run_weather_bot_switch):
    run_weather_bot()

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/nlu/default/weathernlu\entity_synonyms.json'
  "".format(entity_synonyms_file))
Using TensorFlow backend.


Bot loaded. Type a message and press enter: 
hi
Hello! How can I help?
i want to know the weather
In what location?
in london
It is currently Partly cloudy in London at the moment.
